In [1]:
# Author: Kayla Savage & Tapasya Sharma
# Date : January 30th, 2022
# Purpose: File cleans data for INFO 442 project

In [2]:
#Revision Edits: For EDA Purposes - Added fill_mean function to all numeric features

In [3]:
import os
import pandas as pd
import numpy as np

In [4]:
#get the current directory
curDir = os.getcwd()
print(curDir)

c:\Users\taqua\Desktop\2022WINTER


In [6]:
#change to the name of the data file in your folder
df = pd.read_csv("merged_data.csv", encoding = 'latin1')

In [7]:
df.head()

,Date,Symbol,apturnover,arturnover,assetturnover,croic,commontocap,currentratio,ebitdagrowth,ebitdaqoqgrowth,...,profitmargin,roe_simple,stdebttocap,enterprisevalue,country,employees,security_delisted,sic,ipo_price,current_price
0,2020-12-30,RSI,NaN,16.1314,1.6674,NaN,0.231097,0.7344,-4.970785,NaN,...,-0.472693,2.130902,0.000000,663437419.9,United States of America,NaN,FALSE,7990,21.13,9.00
1,2020-12-24,VTAQ,NaN,NaN,NaN,NaN,-0.043767,7.9669,NaN,NaN,...,NaN,0.214396,0.000826,191788580.6,United States of America,4.0,FALSE,6770,10.29,10.07
2,2020-12-23,PHAR,NaN,5.4113,0.5199,-0.325334,0.340220,5.5220,-0.037194,NaN,...,0.230894,0.227771,0.000000,9838328205,Netherlands,258.0,FALSE,2834,19.19,8.79
3,2020-12-23,VHAQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,United States of America,2.0,FALSE,6770,10.15,10.04
4,2020-12-22,XL,7.1506,3.4712,0.1142,NaN,0.998900,38.0183,-3.426013,NaN,...,-2.979939,-0.320843,0.000582,1677155081,United States of America,NaN,FALSE,3714,18.89,2.07


In [8]:
df.shape

(4620, 30)

In [9]:
df = df.dropna(subset = ['Symbol', 'sic'])

In [10]:
def missing_data_ticker(df,threshold):
    
    """
    
    Function to remove rows that have missing data beyond threshold for each ticker 
    
    Parameters:
    
    df - df to be modified
    threshold - threshold in percentage of clean data. Higher the percentage, lesser na values
    
    """
    threshold = df.shape[1] * (threshold/100) 
    df.dropna(thresh = threshold, axis = 0, inplace = True)
    
    return df

In [11]:
def missing_data_column(df,threshold):
    
    """
    
    Function to remove columns that have missing data beyond threshold for each column
    
    Parameters:
    
    df - df to be modified
    threshold - threshold in percentage of clean data. Higher the percentage, lesser na values
    
    """
    threshold = df.shape[0] * (threshold/100) 
    df.dropna(thresh = threshold, axis = 1, inplace = True)

    return df
    

In [12]:
def duplicate_symbols(df,column):
    """
    This function removes any duplicated symbols in the specified column
    
    Parameters:
    
    df - df of which symbols have to removed
    column - Name of column to check
    
    """
    duplicated_data = df[column].duplicated().sum() 
    df.drop_duplicates(subset = column, keep = False, inplace = True)
    
    return df
    
    
    

In [13]:
def null_employee(df,column):
    """
    This function set the column value to O if it is currently null 
    
    Parameters:
    
    df - df to be modified
    column - Name of column to check
    
    """
        
    df.loc[(df[column] == '') | (df[column] == None) | (df[column].isnull()), column] = 0  
    return df
    
    

In [14]:
def current_price(df, col1, col2):
    
    """if the current price column is empty then the security
    delisted is True and set the current price is 0

    Parameters:

    df - df to be modified

    """

    df.loc[(df[col1] == '') | (df[col1] == None) | (df[col1].isnull()), col2] = True  
    df.loc[(df[col1] == '') | (df[col1] == None) | (df[col1].isnull()), col1] = 1
    
    return df



In [15]:
def success_req(df, col1, col2, target_col):
    
    """
    This function checks and modifies requirements for 
    success/failure of target value

    Parameters:

    df - df to be modified
    col1 - First column to check requirement
    col2 - First column to check requirement
    target_col - Target Columm

    """
        
    #STOCK SUCCESS REQUIREMENTS - if delisted then stock success is False
    df.loc[(df[col1] == 0), target_col] = False
    #current price is equal or lower than 50% of the ipo price it is False
    df.loc[(df[col1] == df[col2]), target_col] = False
    #check logic
    df.loc[(df[col1] < df[col2]/ 2), target_col]
    
    return df

In [16]:
def fill_mean(df, col1):
    """
    Parameters: 
    df - df to be modified
    col1 - First column to check and fill mean 
    """
    df[col1] = df[col1].fillna(df.groupby('sic')[col1].transform('mean'))
    
    return df
        

In [17]:
#Adding the Success target variable column - Boolean value (True/False)
NaN = np.nan
df["success"] = NaN

In [18]:
df.dtypes

Date                        object
Symbol                      object
apturnover                  object
arturnover                  object
assetturnover               object
croic                       object
commontocap                float64
currentratio               float64
ebitdagrowth               float64
ebitdaqoqgrowth            float64
efftaxrate                  object
fcffgrowth                 float64
fcffqoqgrowth               object
fcfftointerestex           float64
grossmargin                float64
investedcapitalturnover     object
leverageratio               object
Itdebttocap                 object
nnep                       float64
operatingmargin             object
profitmargin                object
roe_simple                 float64
stdebttocap                float64
enterprisevalue             object
country                     object
employees                  float64
security_delisted           object
sic                         object
ipo_price           

In [19]:
fill_mean(df, "commontocap")
fill_mean(df, "currentratio")
fill_mean(df, "ebitdagrowth")
fill_mean(df, "ebitdaqoqgrowth")
fill_mean(df, "fcffgrowth")
fill_mean(df, "fcfftointerestex")
fill_mean(df, "grossmargin")
fill_mean(df, "nnep")
fill_mean(df, "roe_simple")
fill_mean(df, "stdebttocap")
fill_mean(df, "ipo_price")
fill_mean(df, "current_price")

,Date,Symbol,apturnover,arturnover,assetturnover,croic,commontocap,currentratio,ebitdagrowth,ebitdaqoqgrowth,...,roe_simple,stdebttocap,enterprisevalue,country,employees,security_delisted,sic,ipo_price,current_price,success
0,2020-12-30,RSI,NaN,16.1314,1.6674,NaN,0.231097,0.734400,-4.970785,-0.069725,...,2.130902,0.000000,663437419.9,United States of America,NaN,FALSE,7990,21.130000,9.000000,NaN
1,2020-12-24,VTAQ,NaN,NaN,NaN,NaN,-0.043767,7.966900,-696.592147,1.039494,...,0.214396,0.000826,191788580.6,United States of America,4.0,FALSE,6770,10.290000,10.070000,NaN
2,2020-12-23,PHAR,NaN,5.4113,0.5199,-0.325334,0.340220,5.522000,-0.037194,-20.545706,...,0.227771,0.000000,9838328205,Netherlands,258.0,FALSE,2834,19.190000,8.790000,NaN
3,2020-12-23,VHAQ,NaN,NaN,NaN,NaN,0.139743,9.430820,-696.592147,1.039494,...,-0.094776,-0.104405,NaN,United States of America,2.0,FALSE,6770,10.150000,10.040000,NaN
4,2020-12-22,XL,7.1506,3.4712,0.1142,NaN,0.998900,38.018300,-3.426013,-2.499642,...,-0.320843,0.000582,1677155081,United States of America,NaN,FALSE,3714,18.890000,2.070000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4613,2012-05-10,ADNC,7.1319,13.4547,1.3039,0.966075,1.000000,6.416000,0.737988,0.187756,...,0.106876,0.000000,35349930,United States of America,319.0,TRUE,3674,19.000000,31.749821,NaN
4614,2012-04-27,EDG,2.6535,7.0184,0.3407,-0.021939,0.703960,1.653700,-0.208686,15.081761,...,0.110320,0.004761,8156529071,South Africa,8964.0,FALSE,1040,10.350000,7.535000,NaN
4616,2012-04-18,SDR,NaN,NaN,NaN,NaN,0.583302,4.399244,0.956547,0.450718,...,-0.128106,0.005250,NaN,United States of America,9.0,FALSE,1311,4.683877,25.206667,NaN
4617,2019-02-14,NFE,7.6779,4.8234,0.2075,-0.853604,0.084267,2.064500,-2.423248,-0.075189,...,-2.409280,0.000000,1184744258,United States of America,140.0,FALSE,4924,13.795891,20.970000,NaN


In [20]:
missing_data_ticker(df,20)
missing_data_column(df,20)
duplicate_symbols(df, "Symbol")
null_employee(df, "employees")
current_price(df, "current_price", "security_delisted")
success_req(df, "current_price", "ipo_price", "success")

,Date,Symbol,apturnover,arturnover,assetturnover,croic,commontocap,currentratio,ebitdagrowth,ebitdaqoqgrowth,...,roe_simple,stdebttocap,enterprisevalue,country,employees,security_delisted,sic,ipo_price,current_price,success
0,2020-12-30,RSI,NaN,16.1314,1.6674,NaN,0.231097,0.734400,-4.970785,-0.069725,...,2.130902,0.000000,663437419.9,United States of America,0.0,FALSE,7990,21.130000,9.000000,NaN
1,2020-12-24,VTAQ,NaN,NaN,NaN,NaN,-0.043767,7.966900,-696.592147,1.039494,...,0.214396,0.000826,191788580.6,United States of America,4.0,FALSE,6770,10.290000,10.070000,NaN
2,2020-12-23,PHAR,NaN,5.4113,0.5199,-0.325334,0.340220,5.522000,-0.037194,-20.545706,...,0.227771,0.000000,9838328205,Netherlands,258.0,FALSE,2834,19.190000,8.790000,NaN
3,2020-12-23,VHAQ,NaN,NaN,NaN,NaN,0.139743,9.430820,-696.592147,1.039494,...,-0.094776,-0.104405,NaN,United States of America,2.0,FALSE,6770,10.150000,10.040000,NaN
4,2020-12-22,XL,7.1506,3.4712,0.1142,NaN,0.998900,38.018300,-3.426013,-2.499642,...,-0.320843,0.000582,1677155081,United States of America,0.0,FALSE,3714,18.890000,2.070000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,2012-05-11,IRG,19.3188,65.8114,2.4524,0.019854,0.702414,0.654000,0.170757,0.171005,...,0.082040,0.000000,341348000,NaN,461.0,TRUE,5812,16.850000,50.813333,NaN
4613,2012-05-10,ADNC,7.1319,13.4547,1.3039,0.966075,1.000000,6.416000,0.737988,0.187756,...,0.106876,0.000000,35349930,United States of America,319.0,TRUE,3674,19.000000,31.749821,NaN
4614,2012-04-27,EDG,2.6535,7.0184,0.3407,-0.021939,0.703960,1.653700,-0.208686,15.081761,...,0.110320,0.004761,8156529071,South Africa,8964.0,FALSE,1040,10.350000,7.535000,NaN
4616,2012-04-18,SDR,NaN,NaN,NaN,NaN,0.583302,4.399244,0.956547,0.450718,...,-0.128106,0.005250,NaN,United States of America,9.0,FALSE,1311,4.683877,25.206667,NaN


In [21]:
#saving new file
df.to_csv("preprocessed_data(2).csv", index = False, encoding = 'latin1')